In [1]:
from pyspark import SparkContext, SQLContext, SparkConf, StorageLevel
from collections import namedtuple

In [2]:
sparkconf = SparkConf().setAppName("Uber").setMaster("local[3]")
sc = SparkContext(conf=sparkconf)

24/02/14 19:25:42 WARN Utils: Your hostname, Aditis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
24/02/14 19:25:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/14 19:25:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
uber_data = sc.textFile("/Users/aditi/Desktop/DataEngg/uber_data.csv",4,use_unicode=False).repartition(5)

In [4]:
fields = ('VendorID','tpep_pickup_datetime','tpep_dropoff_datetime','passenger_count',\
          'trip_distance','pickup_longitude','pickup_latitude','RatecodeID','store_and_fwd_flag',\
          'dropoff_longitude','dropoff_latitude','payment_type','fare_amount','extra','mta_tax',\
          'tip_amount','tolls_amount','improvement_surcharge','total_amount')

In [5]:
# uber_data.take(5)
# for reading data

In [6]:
Uber = namedtuple('Uber', fields)
print(Uber)

<class '__main__.Uber'>


In [7]:
def parseRecords(line):
    fields = line.split(",")
    return Uber(fields[0], fields[1], fields[2],\
                fields[3], fields[4], fields[5], \
                fields[6], fields[7], fields[8], \
                fields[9], fields[10], fields[11], \
                fields[12], fields[13], fields[14], \
                fields[15], fields[16], fields[17], fields[18])


In [8]:
uberData = uber_data.map(lambda x: x.decode('utf-8'))
uber = uberData.map(lambda rec : parseRecords(rec) )

In [9]:
uber.take(2)

[Uber(VendorID='2', tpep_pickup_datetime='2016-03-01 00:00:01', tpep_dropoff_datetime='2016-03-01 00:24:06', passenger_count='3', trip_distance='7.18', pickup_longitude='-73.98577880859380', pickup_latitude='40.74119186401370', RatecodeID='1', store_and_fwd_flag='N', dropoff_longitude='-73.94635009765630', dropoff_latitude='40.79787826538090', payment_type='1', fare_amount='23.5', extra='0.5', mta_tax='0.5', tip_amount='3.2', tolls_amount='0.0', improvement_surcharge='0.3', total_amount='28.0'),
 Uber(VendorID='2', tpep_pickup_datetime='2016-03-01 00:00:01', tpep_dropoff_datetime='2016-03-01 00:02:03', passenger_count='2', trip_distance='0.54', pickup_longitude='-73.9884262084961', pickup_latitude='40.76416015625', RatecodeID='1', store_and_fwd_flag='N', dropoff_longitude='-73.99239349365230', dropoff_latitude='40.75822448730470', payment_type='2', fare_amount='4.0', extra='0.5', mta_tax='0.5', tip_amount='0.0', tolls_amount='0.0', improvement_surcharge='0.3', total_amount='5.3')]

In [10]:
def isfloat(str):
    try:
        float(str)
        return True
    except ValueError:
        return False

In [11]:
passenger_30 = uber.filter(lambda rec : isfloat(rec.trip_distance) and float(rec.trip_distance) < 10.00)
passengers = passenger_30.map(lambda rec: (rec.total_amount, rec.trip_distance, rec.passenger_count))
passengers.take(5)

[('28.0', '7.18', '3'),
 ('5.3', '0.54', '2'),
 ('9.3', '1.7', '1'),
 ('9.0', '1.1', '1'),
 ('12.36', '2.1', '1')]

In [12]:
total_P = uber.filter(lambda rec : rec.passenger_count.isdigit() and isfloat(rec.total_amount) and \
                      isfloat(rec.trip_distance) and (float(rec.total_amount) > 30.00) & (float(rec.trip_distance)<10.00) \
                      & (int(rec.passenger_count) > 3)).map(lambda rec: (int(rec.passenger_count), float(rec.total_amount)))
total = total_P.reduceByKey(lambda a,b: a+b).sortBy(lambda rec: rec[0], ascending = True)


In [13]:
total.take(5)

[(4, 2279.4300000000003), (5, 15041.870000000008), (6, 8635.969999999998)]

In [14]:
same_dest = uber.filter(lambda rec: rec.pickup_longitude == rec.dropoff_longitude and rec.pickup_latitude == rec.dropoff_latitude).map(lambda rec: (rec[5], rec[6], rec[9], rec[10])).map(lambda rec: (rec, 1)).reduceByKey(lambda a,b: a+b).sortBy(lambda rec: rec[1], ascending = False)
same_dest.take(5)
#Counts the no of trips with same destination as start point 

[(('0.0', '0.0', '0.0', '0.0'), 818),
 (('-73.93831634521480',
   '40.747291564941400',
   '-73.93831634521480',
   '40.747291564941400'),
  21),
 (('-74.10514068603520',
   '40.68149185180660',
   '-74.10514068603520',
   '40.68149185180660'),
  20),
 (('-73.75502014160160',
   '40.71524810791020',
   '-73.75502014160160',
   '40.71524810791020'),
  20),
 (('-73.98658752441410',
   '40.76255035400390',
   '-73.98658752441410',
   '40.76255035400390'),
  20)]

In [15]:
total_same_dest = same_dest.map(lambda a: (1, a[1])).reduceByKey(lambda a,b: a+b).map(lambda rec: (rec[1])).collect()
print(total_same_dest)

[1272]


In [16]:
sc.stop()